# **Initial setup**

Install Bambu and required packages:

In [ ]:
!echo "deb http://ppa.launchpad.net/git-core/ppa/ubuntu $(cat /etc/os-release | grep UBUNTU_CODENAME | sed 's/.*=//g') main" >> /etc/apt/sources.list.d/git-core.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A1715D88E1DF1F24
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git verilator wget
!wget https://release.bambuhls.eu/appimage/bambu-latest.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!ln -sf $PWD/bambu-*.AppImage /bin/tree-panda-gcc
!git clone --depth 1 --filter=blob:none --branch doc/bambu101 --sparse https://github.com/ferrandi/PandA-bambu.git
%cd PandA-bambu
!git sparse-checkout set documentation/bambu101
%cd ..
!mv PandA-bambu/documentation/bambu101/* .

# What does High-Level Synthesis do?
The High-Level Synthesis process transforms an high-level language description into an hardware design. The input representation can be any high-level language supported by the HLS tool (i.e.: C/C++,FORTRAN,SystemC) which takes care of both optimization and translation to the output RTL-level language (i.e.: Verilog, VHDL).

The flow is similar to that of a standard compiler like GCC/Clang while after preprocessing and generic optimization of the input description further hardware-oriented transformations are applied to the code. Finally, the backend takes care of translating the code into an hardware design instantiating functional units, generating a schedule for the operations, and binding each operation to a functional unit.

## Input application
The HLS tool requires an input description of the application to be translated into hardware: it can be any type of application given it has a single top level function which handles all the inputs and outputs.

As an example an implementation of a CRC kernel is used: /content/basic_usage/icrc.c

## Minimal set of parameters
The HLS tool also requires some specification about the expected output interface to be generated.

### Top level interface
As a standard compiler needs a main function to know where to start, the HLS tool needs a top level interface to be used as the interface of the generated hardware module.
Following the CRC example, this can be specified through the --top-fname=icrc1 command line option. This will let the HLS tool know the hardware design interface has to be the same of function icrc1 from the input representation.

### Target hardware device
The HLS tool needs to know what is the hardware target for which the RTL description needs to be generated since different assumptions are made based on the target device. 
For this example a Xilinx FPGA board has been selected (xc7z020-1clg484) and is required through the --device-name=xc7z020-1clg484-VVD command line option.
Furthermore, the required clock period of the generated accelerator needs to be specified throgh the --clock-period option. For a target frequency of 200MHz --clock-period=5 has to be used, as an example.

### Generated design verification
Bambu HLS offers the possibility to generate some additional code to run a verification of the output design with respect to the input representation. This can be required by passing an xml file where the top level interface parameters for the test are specified.
This is shown in the following: each testbench entry contains the two parameters of the icrc1 function.
The file is passed to the tool through --generate-tb=test.icrc1.xml.
Output values are automatically gathered from live execution of the input code and stored into the generated verification IP. 

In [ ]:
%%writefile /content/basic_usage/test.icrc1.xml
<?xml version="1.0"?>
<function>
   <testbench crc="6452" onech="5" />
   <testbench crc="942856" onech="3" />
</function>

## Generating the accelerator
We are now ready to generate the hardware design of the icrc1 function using the following:

In [ ]:
%cd /content/basic_usage 
!bambu icrc1.c --top-fname=icrc1 --device-name=xc7z020-1clg484-VVD --clock-period=5 --generate-tb=test.icrc1.xml --simulate |& tee log.txt

# Output hardware design
After the High-Level Synthesis process has completed successfully, the user can find the generated RTL design in the /content/basic_usage/icrc1.v file.

The HLS tool also generates a synthesis script /content/basic_usage/synthesize_Synthesis_icrc1.sh along with the Verilog description so that it can be used to run the vendor specific synthesis tool to generate the bitstream for the target FPGA.